# OD Backup controller
This should develop into the controller run a backup automatically.  Using CDXPBUrner for drive control.

- Aim read drive and convert to a stack of disks eg DVD
- Want to verify quality

## CDBurnerXP Driver Controller

In [1]:
import subprocess


class ODMedia:
    def __init__(self):
        self.path_root = r"C:\Program Files (x86)\CDBurnerXP\cdbxpcmd.exe"
    
    def version(self):
        result = subprocess.run([self.path_root, "--version"], stdout=subprocess.PIPE)
        return result.stdout.decode('utf-8')

    def list_drives(self):
        result = subprocess.run([self.path_root, "--list_drives"], stdout=subprocess.PIPE
        )
        print(result.stdout.decode('utf-8').split('\n'))
        return result.stdout.decode('utf-8')

    @property
    def number_of_drives(self):
        result = self.list_drives()
        if result.find('\n') == -1:
            return 1  # Todo check for 0
        else:
            return len(result.split('\n'))

    def disk_open(self):
        result = subprocess.run([self.path_root, "--eject", "-device:0"], stdout=subprocess.PIPE)
        return result.stdout.decode('utf-8')

    def disk_close(self):
        result = subprocess.run([self.path_root, "--load", "-device:0"], stdout=subprocess.PIPE)
        return result.stdout.decode('utf-8')

    def burn_disk(self, file_name):
        """ eg"""
        result = subprocess.run([self.path_root, "--burn-iso", "-device:0",
                                f'-file:{file_name}'], stdout=subprocess.PIPE)
        print(result)
        return result.stdout.decode('utf-8')
    

media = ODMedia()

## Robot controller

In [2]:
import time

import od_control

robot = od_control.ODRobot(media.disk_open, media.disk_close, com_port="COM6")

Robot>import control1
>>> 


## DVD  control and burning

In [3]:
class ODStatus:
    """Reads the status of the optical disk in the drive.
    Initialisation reads all the state information and then you can use the data as necessary"""
    def __init__(self, media):
        # Parse media information
        self.number_of_drives = len(media.list_drives().split('\n')[2:-4])
        # test = media.disk_type()
        test = "Todo"
        if test == '\rThere was an error obtaining disc info!\r\n':
            self.disk_loaded = False
        else:
            self.disk_loaded = True
            """Parses disk type into a json object"""
            lines = test.split('\n')
            print(lines)
            description=lines[0].split(',')  # First line
            self.media_type = description[0]
            self.free_space = int(description[2].strip().split(' ')[0]) * 2048
            self.capacity = int(lines[3].strip().split(':')[1].strip()) * 2048
            self.disk_is_blank = self.free_space == self.capacity and self.free_space != 0

    def __str__(self):
        result = f"Number of drives = {self.number_of_drives}\n"
        if self.disk_loaded:
            result += f"Disk Loaded\n"
            result += f"Size = {self.capacity}\n"
            result += f"Free space = {self.free_space}\n"
            if self.disk_is_blank:
                result += f"Blank disk\n"
            else:
                #result += f"{self.readme}\n"
                result += f"Media type = {self.media_type}\n"

        else:
            result += f"No Disk Loaded\n"
        return result

    def _readme(self):
        # Assume disk loaded
        # look for readme.markdown and then print
        with open("d:/readme.markdown","r") as f:
            content = f.readlines()
        return content


a = ODStatus(media)
print(a)


['0: PIONEER  BD-RW   BDR-S09  (D:\\)\r', '']
['Todo']


IndexError: list index out of range

## Sort disks into blank and used
This will take n disks and read them.  If blank they will be put in out bin otherwise a waste place.

In [ ]:
class ODError(Exception):
    pass

for i in range(9):  
    # Check if drive empty
    a = ODStatus(media)
    if not a.disk_loaded:
        robot.load()
        a = ODStatus(media)
        print(a)
    if not a.disk_loaded:
        raise ODError(f'On cycle {i} not disk was loaded and it should have been')
    if a.disk_is_blank:
        print(f'Disk {i} is blank')
        robot.unload()
    else:
        print(f'Disk {i} has data')
        robot.unload(destination_bin='waste')

In [ ]:
# refill in bin
for i in range(9):  
    robot.unload(source_bin='out', destination_bin='in')

# A Job
- Get a list of dirs (ca 5GB)  (first time will be just two dirs as want to improve disc identification)
- Create and archive
- create and burn iso's

TODO use pyarchive as module

In [ ]:
from functools import partial
import os
import re
import shutil

root_path = r'Z:\Home Pictures'
number_start = re.compile('^\d\d\d\d')

def getFolderSize(start_path = '.'):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size

size_sum = 0
results = []
with os.scandir(root_path) as it:
    for entry in it:
        if not entry.is_file():
            if number_start.search(entry.name):
                print(entry.path)
                folder_size = getFolderSize(entry.path) 
                size_sum += folder_size
                results.append([entry.path, folder_size, size_sum])
print(results)
for i,list in enumerate(results):
    if list[2] > 1.5*1E9:
        print(i, list)
        break
# Get the list of directories
target_dirs = [d[0] for d in results[:i]]
print(target_dirs)

In [5]:
import os
from pathlib import Path

from odarchive import Archiver, load_archiver_from_json
#from h3timeit import h3_timeit


In [9]:
"""
This should make for easier debugging
It creates a single archive and prints out the information for it.
On the Z drive
timeit started 2018-06-18 11:15:25.988937
Initializing file database
Number of files = 40,954
Data size       = 228,605,334,662
Is segmented    = False
Number of dirs  = 1458
Max dir depth   = 11 (on source file system)
 Dir =: Z:\Home Pictures\H3 to Sort\2012-06-29\ManualArchive\Apalone\Quotes\healthMan\awstats\awstats\icon\flags

'test_scan_Z_drive'  436541.60 ms

"""


def test_create_job():
    # Archive part
    ar = Archiver()
    ar.create_file_database(Path("Z:\\Home Pictures\\2000"))
    ar.save()
    ar.convert_to_hash_database(verbose = True)
    ar.hash_db.save()  #  Creates catalogue.json
    ar.save()
    # ar = load_archiver_from_json()  only once catalogue.json has been crate
    ar.segment("cd")
    ar.save()
    ar.hash_db.save()
    print(ar.file_db.get_info())

def test_create_some_iso():
    ar = load_archiver_from_json()  #only once catalogue.json has been crate
    print(ar.get_info())
    for n in range(ar.last_disc_num):
        ar.write_iso(disc_num=n)
        filename = f"new_{n:04}.iso"
        print(f'Written {filename}')

def test_burn_some_pictures():
    # Check and make sure drive is empty
    a = ODStatus(media)
    if a.disk_loaded:
        print(f'Drive had disk in which is being put into waste bin')
        robot.unload(destination_bin='waste') 
    # Archive part
    ar = Archiver()
    ar.create_file_database(Path("Z:\\Home Pictures\\2000"))
    ar.save()
    ar.convert_to_hash_database(verbose = True)
    ar.hash_db.save()  #  Creates catalogue.json
    ar.save()
    # ar = load_archiver_from_json()  only once catalogue.json has been crate
    ar.segment("cd")
    ar.save()
    ar.hash_db.save()
    print(ar.file_db.get_info())
    for n in range(ar.last_disc_num):
        #ar = load_archiver_from_json()
        ar.write_iso(disc_num=n)
        # TODO make filename part of archiver
        filename = f"new_{n:04}.iso"
        # Load drive and burn disc
        robot.load()
        a = ODStatus(media)
        print(a)
        if a.disk_is_blank:
            media.burn_disk(filename)
            robot.unload()
        else:
            raise ODError(f'Cycle {n} and had non blank disk')

# test_create_job() First part
test_create_some_iso() # second part
#test_burn_some_pictures() # first second and third parts


Number of entries = 97
Data size       = 2,176,068,308 bytes
Is segmented    = True
  Disc segment size = cd, 737,280,000 bytes
  Catalogue size = 32,768 bytes
  Number of discs = 2
Number of files = 194
  Largest file  = 546,299,752
Number of dirs  = 6
Max dir depth   = 2 (on source file system)
 Dir =: /DATA/2000-12-28

Disc num = |0|


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'new_0000.iso'

In [7]:
## If failed but have archive then use this code
def test_burn_isos():
    # Check and make sure drive is empty
    #a = ODStatus(media)
    #if a.disk_loaded:
    #    print(f'Drive had disk in which is being put into waste bin')
    #    robot.unload(destination_bin='waste') 
    # Archive part
    ar = load_archiver_from_json()  # Assume segmented and ready to burn
    print(ar.file_db.get_info())
    for n in range(ar.last_disc_num+1):
        #ar = load_archiver_from_json()
        ar.write_iso(disc_num=n)
        # TODO make filename part of archiver
        filename = f"new_{n:04}.iso"
        # Load drive and burn disc
        robot.load()
        media.burn_disk(filename)
        #a = ODStatus(media)
        #print(a)
        #if a.disk_is_blank:
        #    media.burn_disk(filename)
        #    robot.unload()
        #else:
        #    raise ODError(f'Cycle {n} and had non blank disk')

        
test_burn_isos()

Number of entries = 102
Data size       = 1,088,035,534 bytes
Number of dirs  = 6
Max dir depth   = 4 (on source file system)
 Dir =: Z:\Home Pictures\2000\2000-12-28\Xmas 00

Disc num = |0|
Robot>control1.th.release()
>>> 
Robot>control1.z_axis.nudge(51,run_time=150)
Shutdown Z Axis nudge
>>> 
Robot>control1.th.grip()
>>> 
Robot>control1.z_axis.nudge(51,run_time=40)
Shutdown Z Axis nudge
>>> 
Robot>control1.th.release()
>>> 
CompletedProcess(args=['C:\\Program Files (x86)\\CDBurnerXP\\cdbxpcmd.exe', '--burn-iso', '-device:0', '-file:new_0000.iso'], returncode=0, stdout=b'Starting to burn ISO image from new_0000.iso...\r\n0%\r\n1%\r\n2%\r\n3%\r\n4%\r\n5%\r\n6%\r\n7%\r\n8%\r\n9%\r\n10%\r\n11%\r\n12%\r\n13%\r\n14%\r\n15%\r\n16%\r\n17%\r\n18%\r\n19%\r\n20%\r\n21%\r\n22%\r\n23%\r\n24%\r\n25%\r\n26%\r\n27%\r\n28%\r\n29%\r\n30%\r\n31%\r\n32%\r\n33%\r\n34%\r\n35%\r\n36%\r\n37%\r\n38%\r\n39%\r\n40%\r\n41%\r\n42%\r\n43%\r\n44%\r\n45%\r\n46%\r\n47%\r\n48%\r\n49%\r\n50%\r\n51%\r\n52%\r\n53%\r\n54

In [ ]:
#Test
robot.load(source_bin='out')

In [10]:
robot.disk_close()

'Closing tray...\r\n'

In [11]:
# Reject disk and load new blank
robot.unload() #(destination_bin='waste')
#robot.load()

Robot>control1.th.release()
>>> 
Robot>control1.z_axis.nudge(50,run_time=70)
Shutdown Z Axis nudge
>>> 
Robot>control1.th.grip()
>>> 
Robot>control1.z_axis.nudge(50,run_time=20)
Shutdown Z Axis nudge
>>> 
Robot>control1.th.release()
>>> 


## Temporary playpen


In [ ]:
robot.disk_close()
a = ODStatus(media)
print(a)